In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
np.set_printoptions(precision=128)

functionCalculated = 0

def f(x):
    x1 = x[0]
    x2 = x[1]
    global functionCalculated
    functionCalculated += 1
    # return x1**2 + x2**2
    # return (x1-1)**2 + 100*(x2-1)**2
    return 100*(x1**2-x2)**2 + (x1-1)**2
    # return (100*(x1-x2)**2 + (x1-1)**2)
    # return (10*(x1-x2)**2 + (x1-1)**2)**4

In [ ]:
# print(f"{h}, {h*np.array([1, 0])}, {h*np.array([0, 1])}")
def fderright(x, fx, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - fx) / h
    gradFx2 = (f(x + h*np.array([0, 1])) - fx) / h
    return np.array([gradFx1, gradFx2])

def fderleft(x, fx, h):
    gradFx1 = (fx - f(x - h*np.array([1, 0]))) / h
    gradFx2 = (fx - f(x - h*np.array([0, 1]))) / h
    return np.array([gradFx1, gradFx2])

def fdercenter(x, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - f(x - h*np.array([1, 0]))) / (2*h)
    gradFx2 = (f(x + h*np.array([0, 1])) - f(x - h*np.array([0, 1]))) / (2*h)
    return np.array([gradFx1, gradFx2])

def fder(x, fx, h, type):
    if type == 'right':
        return fderright(x, fx, h)
    if type == 'left':
        return fderleft(x, fx, h)
    if type == 'center':
        return fdercenter(x, h)
    else:
        raise Exception('Not valid type')
    

In [ ]:
def norm(x):
    x1 = x[0]
    x2 = x[1]
    return math.sqrt(x1**2 + x2**2)

def stopCriteria1(xk, fxk, xk_1, fxk_1, e):
    if ((norm(xk_1 - xk) / norm(xk)) <= e) and (math.fabs(fxk_1 - fxk) <= e):
        return True
    return False

def stopCriteria2(gradFxk, e):
    if (norm(gradFxk) <= e):
        return True
    return False

In [ ]:
# Code for finding optimal Lambda
functionCalculatedForLambda = 0

svenCoefLambda = 0.01 # 0.00001 # 0.000098
l0 = 0

eGoldenCut = 0.001 # 0.1 # 0.11
eDSKPauela = 0.001

'''
def f(x):
    x1 = x[0]
    x2 = x[1]
    return 3*(x1-15)**  2 -x1*x2 +4*x2**2
''' 

def FLambda(xk, l, Sk):
    global functionCalculatedForLambda
    functionCalculatedForLambda += 1
    return f(xk + l*Sk)

# trustInterval = sven(np.array([-22, -22]), np.array([0, 1]))
# print(goldenCut(trustInterval, np.array([-22, -22]), np.array([0, 1])))
# print(functionCalculatedForLambda)
    

In [ ]:
def sven(xk, Sk):
    deltaL = svenCoefLambda * (norm(xk) / norm(Sk))
    FLambda0 = FLambda(xk, l0, Sk)
    FLambdaPlus = FLambda(xk, l0+deltaL, Sk)
    FLambdaMinus = FLambda(xk, l0-deltaL, Sk)
    
    coef = 0
    currentFValue = 0
    if FLambda0 < FLambdaPlus and FLambda0 < FLambdaMinus:
        print("Sven: both Lambdas are increasing funcion")
        return np.array([-deltaL, deltaL])
    if FLambda0 > FLambdaPlus and FLambda0 > FLambdaMinus:
        raise Exception("Sven: both Lambdas are deacreasing function")
    if FLambda0 > FLambdaPlus:
        coef = 1
        currentFValue = FLambdaPlus
        l1 = l0 + coef * deltaL
    if FLambda0 > FLambdaMinus:
        coef = -1
        currentFValue = FLambdaMinus
    l1 = l0 + coef * deltaL
    currentlValue = l1
    lambdaValues = [l0, l1]
    FNewLambda = 0
    n = 1
    while True:
        newLambda = currentlValue + 2*n*coef * deltaL
        lambdaValues.append(newLambda)
        FNewLambda = FLambda(xk, newLambda, Sk)
        if (currentFValue < FNewLambda):
            break
        currentlValue = newLambda
        currentFValue = FNewLambda
        n = 2*n
    print(f"SVEN - deltaL: {deltaL}, Lambdas: {lambdaValues}, functionCalculatedForLambda: {functionCalculatedForLambda}")
    return np.array([lambdaValues[-3], (lambdaValues[-2] + lambdaValues[-1]) / 2])

In [ ]:
def goldenCut(lTrustInterval, xk, Sk):
    a = lTrustInterval[0]
    b = lTrustInterval[1]
    biggerCut = (math.sqrt(5) - 1) / 2
    smallerCut = 1 - biggerCut
    L = b - a
    x1 = a + smallerCut * L
    x2 = a + biggerCut * L
    Fx1 = FLambda(xk, x1, Sk)
    Fx2 = FLambda(xk, x2, Sk)

    while (L > eGoldenCut):
        if Fx1 < Fx2:
            b = x2
            x2 = x1
            L = b - a
            x1 = a + smallerCut * L
            Fx2 = Fx1
            Fx1 = FLambda(xk, x1, Sk)
        else:
            a = x1
            x1 = x2
            L = b - a
            x2 = a + biggerCut * L
            Fx1 = Fx2
            Fx2 = FLambda(xk, x2, Sk)
    print(f"GOLDENCUT - INTERVAL: {np.array([x1, x2])}, functionCalculatedForLambda: {functionCalculatedForLambda}")
    return np.array([x1, x2])

In [ ]:
def dskStopCriteria(x2, Fx2, x_new, Fx_new):
    if (math.fabs(Fx2 - Fx_new) <= eDSKPauela) and (math.fabs(x2 - x_new) <= eDSKPauela):
        return True
    return False


# NOTE: OPTIMIZE DSKPAUELA
def DSKPauela(lTrustInterval, xk, Sk):
    # First iteration
    x1 = lTrustInterval[0]
    x2 = (lTrustInterval[0] + lTrustInterval[1]) / 2
    x3 = lTrustInterval[1]

    Fx1 = FLambda(xk, x1, Sk)
    Fx2 = FLambda(xk, x2, Sk)
    Fx3 = FLambda(xk, x3, Sk)

    deltaX = x2 - x1
    x_new = x2 + (deltaX * (Fx1 - Fx3)) / (2*(Fx1 - 2*Fx2 + Fx3))

    i = 0
    while True:
        i += 1
        Fx_new = FLambda(xk, x_new, Sk)

        print(f"x1: {x1}, x2: {x2}, x3: {x3}, Fx1: {Fx1}, Fx2: {Fx2}, Fx3: {Fx3}")
        print(f"x_new: {x_new}, x2: {x2}, Fx_new:{Fx_new}, Fx2:{Fx2}")

        if dskStopCriteria(x2, Fx2, x_new, Fx_new):
            print(f"DSKPAUELA - Lambda: {x_new}, functionCalculatedForLambda: {functionCalculatedForLambda}")
            return x_new
        
        if Fx_new < Fx2:
            if x1 < x_new and x_new < x2:
                x1 = x1
                x3 = x2
                x2 = x_new

                Fx1 = Fx1
                Fx2 = Fx_new
                Fx3 = Fx2
            elif x2 < x_new and x_new < x3:
                x1 = x2
                x3 = x3
                x2 = x_new

                Fx1 = Fx2
                Fx2 = Fx_new
                Fx3 = Fx3
            else:
                raise Exception(f"DSKPauela: can't choose points: x1: {x1} x2: {x2}, x3: {x3}, x_new: {x_new}, iteration: {i}")
        elif Fx_new > Fx2:
            if x_new > x2:
                x1 = x1
                x2 = x2
                x3 = x_new

                Fx1 = Fx1
                Fx2 = Fx2
                Fx3 = Fx_new
            
            elif x_new < x2:
                x1 = x_new
                x2 = x2
                x3 = x3 

                Fx1 = Fx_new
                Fx2 = Fx2
                Fx3 = Fx3
            else:
                raise Exception(f"Equal points") 
        else:
            raise Exception(f"Equal points")

        a1 = (Fx2 - Fx1) / (x2 - x1)
        a2 = (1 / (x3 - x2)) * ((Fx3 - Fx1) / (x3 - x1) - (Fx2 - Fx1) / (x2 - x1))
        x_new = (x1 + x2) / 2 - a1 / (2*a2)

In [ ]:
def adam(m, v, l, Sk, i, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    m = beta1 * m + (1 - beta1) * Sk
    v = beta2 * v + (1 - beta2) * (Sk ** 2)

    m_hat = m / (1 - beta1 ** i)
    v_hat = v / (1 - beta2 ** i)  
    
    l = l - alpha * m_hat / (np.sqrt(v_hat) + epsilon)
    return m, v, l


In [ ]:
import time

def MSD(x0, h, e, derType='center', l_type='opt', mop_type='gc'):
    i = 0
    xk = x0
    Fxk = f(xk)
    gradFxk = fder(xk, Fxk, h, type=derType)

    m = np.zeros_like(xk)
    v = np.zeros_like(xk)
    l = 0

    while True:
        i += 1
        print(f"\nITERATION NUMBER {i}")
        Sk = - gradFxk

        if l_type == 'opt':
            if mop_type == 'gc':
                l = goldenCut(sven(xk, Sk), xk, Sk)
            elif mop_type == 'dsk':
                l = DSKPauela(sven(xk, Sk), xk, Sk)
            else:
                raise Exception('Not possible mop_type')
            xk_new = xk + l * Sk
        
        elif l_type == 'const':
            m, v, l = adam(m, v, l, gradFxk, i)
            xk_new = xk + l * Sk / norm(Sk)
        
        else:
            raise Exception('Not possible l_type')
        
        Fxk_new = f(xk_new)
        
        gradFxk_new = fder(xk_new, Fxk_new, h, type=derType)
        print(f'Xk: {xk}, Xk+1: {xk_new}, Lambda: {l}, Fxk: {Fxk}, Fxk+1: {Fxk_new}, GradFxk: {gradFxk}, GradFxk+1: {gradFxk_new}')
        
        if stopCriteria1(xk, Fxk, xk_new, Fxk_new, e): # stopCriteria2(gradFxk_new, e): # stopCriteria1(xk, Fxk, xk_new, Fxk_new, e):
            return xk_new
        xk = xk_new
        Fxk = Fxk_new
        gradFxk = gradFxk_new

        if functionCalculated > 600000:
            print('EXITED: TOO MUCH OPEARTIONS')
            time.sleep(3)
            return xk

x0 = np.array([-1.2, 0])
h = 0.000000001
e = 0.001
eDSKPauela = 0.001
eGoldenCut = 0.001
svenCoefLambda = 0.01
functionCalculated = 0
functionCalculatedForLambda = 0
res = MSD(x0, h, e, derType='center', l_type='opt', mop_type='dsk')
print(functionCalculated)
print(np.round(res, 5))
print(round(f(res), 5))

In [ ]:
start = 0.01
stop = 0.000000000000001
step = 0.1
array = []
while start >= stop:
    array.append(start)
    start *= step
h_values = np.array(array)

h_values = np.array([0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001,
                     0.0000000001, 0.00000000001, 0.000000000001, 0.0000000000001, 0.00000000000001,
                     0.000000000000001])

e = 0.001
rightMinimum = []
leftMinimum = []
centerMinimum = []

rightFcalc = []
leftFcalc = []
centerFcalc = []
print(h_values)
global functionCalculated
for h in h_values:
    functionCalculated = 0
    res = MSD(x0, h, e, derType='center')
    centerMinimum.append(res)
    centerFcalc.append(functionCalculated)


for h in h_values:
    functionCalculated = 0
    res = MSD(x0, h, e, derType='right')
    rightMinimum.append(res)
    rightFcalc.append(functionCalculated)

for h in h_values:
    functionCalculated = 0
    res = MSD(x0, h, e, derType='left')
    leftMinimum.append(res)
    leftFcalc.append(functionCalculated)
    
    

In [ ]:
# Build plots
plt.figure(figsize=(16, 8))
plt.semilogx(h_values, rightFcalc, label='Правостороння')
plt.semilogx(h_values, leftFcalc, label='Лівостороння')
plt.semilogx(h_values, centerFcalc, label='Центральна')

plt.xticks(h_values)
# Adding a legend and labels
plt.legend()
plt.xlabel('h')
plt.ylabel('Кількість обчислень функції')
plt.gca().invert_xaxis()
# Displaying the plot
plt.show()

In [ ]:
resultsRigthMiminum = list(map(lambda x: f(x), rightMinimum))
resultsLeftMiminum = list(map(lambda x: f(x), leftMinimum))
resultsCenterMiminum = list(map(lambda x: f(x), centerMinimum))

plt.figure(figsize=(16, 8))
plt.semilogx(h_values, resultsRigthMiminum, label='Правостороння')
plt.semilogx(h_values, resultsLeftMiminum, label='Лівостороння')
plt.semilogx(h_values, resultsCenterMiminum, label='Центральна')

plt.xticks(h_values)
plt.yticks([0.001, 0.01, 0.01, 0.025, 0.05, 0.075, 0.1, 0.3])
# Adding a legend and labels
plt.legend()
plt.xlabel('h')
plt.ylabel('Значення функції')
plt.gca().invert_xaxis()
# Displaying the plot
plt.show()

In [ ]:
eMOPvalues = np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001])
global functionCalculated

gcMinimum = []
dskMinimum = []

gcFCalc = []
dskFCalc = []

h = 0.000000001
e = 0.001

for e in eMOPvalues:
    eGoldenCut = e
    functionCalculated = 0
    res = MSD(x0, h, e, derType='center', l_type='opt', mop_type='gc')
    gcMinimum.append(res)
    gcFCalc.append(functionCalculated)

for e in eMOPvalues:
    eDSKPauela = e
    functionCalculated = 0
    res = MSD(x0, h, e, derType='center', l_type='opt', mop_type='dsk')
    dskMinimum.append(res)
    dskFCalc.append(functionCalculated)

In [ ]:

resultsGCMiminum = list(map(lambda x: f(x), gcMinimum))
resultsDSKMiminum = list(map(lambda x: f(x), dskMinimum))

plt.figure(figsize=(16, 8))
plt.semilogx(eMOPvalues[5:], resultsGCMiminum[5:], label='Золотий переріз')
plt.semilogx(eMOPvalues[5:], resultsDSKMiminum[5:], label='ДСК Пауелла')

plt.xticks(eMOPvalues[5:])
# plt.yticks([0.001, 0.01, 0.01, 0.025, 0.05, 0.075, 0.1, 0.3])
# Adding a legend and labels
plt.legend()
plt.xlabel('e для МОП')
plt.ylabel('Значення функції')
plt.gca().invert_xaxis()
# Displaying the plot
plt.show()

In [ ]:
# Build plots
plt.figure(figsize=(16, 8))
plt.semilogx(eMOPvalues, gcFCalc, label='Золотий переріз')
plt.semilogx(eMOPvalues, dskFCalc, label='ДСК Пауелла')

plt.xticks(eMOPvalues)
# Adding a legend and labels
plt.legend()
plt.xlabel('h')
plt.ylabel('Кількість обчислень функції')
plt.gca().invert_xaxis()
# Displaying the plot
plt.show()

In [ ]:
print(eMOPvalues)
print(np.around(gcMinimum, 5))
print(np.around(resultsGCMiminum, 8))
print(gcFCalc)

In [ ]:
print(np.around(dskMinimum, 5))
print(np.around(resultsDSKMiminum, 8))
print(dskFCalc)